In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import pickle
import json
import seaborn as sns



def base_results_graph(predictions_base, dataset, domain):

        if domain == 'source':
            true_measurements_unscaled = (scaler_source.inverse_transform(y_test_source))*(sqm_source/100)
        elif domain == 'target':
            true_measurements_unscaled = (scaler_target.inverse_transform(y_test_target))*(sqm_target/100)
        elif domain == 'target_2':
            true_measurements_unscaled = (scaler_target_2.inverse_transform(y_test_target_2))*(sqm_target_2/100)
        
        periods = [24, 7*24, 30*24]
        time_periods = ['Day','Week','Month']
        

        for period,time_period in zip(periods,time_periods):

            # Create a time index for the next 24 hours
            time_index = range(period)
            plt.figure(figsize=(20, 9))

            # Plot predicted values
            plt.plot(time_index, predictions_base[0:period], label='Predicted_base_{}'.format(domain), color = 'blue')

            # Plot true measurements
            plt.plot(time_index, true_measurements_unscaled[0:period], label='True', color = 'orange', linestyle='--', alpha = 0.6)

            # Add labels and title
            plt.xlabel('Time (hours)')
            plt.ylabel('Measurement')
            plt.title('CNN Base model next {} predictions / true measurements in {} domain'.format(time_period, domain))
            plt.legend()

            # Save plot
            plt.savefig('Results/Naive/{}/Next {} Results/Prediction Results/BASE_{}_results.png'.format(dataset,time_period,domain), dpi=500)

            # Show plot
            plt.show()



def error_ranges(predictions_base, dataset, domain):

    if domain == 'source':
        true_measurements_unscaled = (scaler_source.inverse_transform(y_test_source))*(sqm_source/100)
    elif domain == 'target':
        true_measurements_unscaled = (scaler_target.inverse_transform(y_test_target))*(sqm_target/100)
    elif domain == 'target_2':
        true_measurements_unscaled = (scaler_target_2.inverse_transform(y_test_target_2))*(sqm_target_2/100)
    
    
    periods = [24, 7*24, 30*24]
    time_periods = ['Day','Week','Month']

    for period,time_period in zip(periods,time_periods):

        base_errors = np.abs(predictions_base[0:period] - true_measurements_unscaled[0:period])
        base_errors = base_errors.ravel()

        error_data = {
            'Without TL': base_errors 
        }

        # Create the boxplot without fill colors
        plt.figure(figsize=(15, 9))
        boxplot = sns.boxplot(data=list(error_data.values()), showfliers=False, widths=0.3, color='whitesmoke')

        # Customize x-ticks
        plt.xticks(ticks=np.arange(len(error_data.keys())), labels=error_data.keys())

        # Remove fill color for boxes (no palette)
        for i, box in enumerate(plt.gca().artists):
            box.set_facecolor('grey')  # No fill color
            box.set_edgecolor('black')  # Black borders
            box.set_linewidth(2.5)  # Set line width

        medians = []
        medians.append(boxplot.lines[4])

        # Customize median line to be orange and thicker
        for median in medians:  # The first 4 lines are whiskers, the next 4 are medians
            median.set_color('gold')
            median.set_linewidth(2.3)  # Thicker median line

        # Add title and labels
        plt.title("Naive model error spread for next {} predictions in {} {} domain".format(time_period,dataset,domain), fontsize=14)
        plt.ylabel("error")
        
        plt.grid()

        # Save plot
        plt.savefig('Results/Naive/{}/Next {} Results/Error Range Results/Error_ranges_{}.png'.format(dataset,time_period,domain), dpi=500)

        # Show the plot
        plt.show()





def metrics_table(dataset):

    range = [0,4,8]
    time_periods = ['Day','Week','Month'] 

    for i,time_period in zip(range,time_periods):

        if dataset == '12M':

            RMSE_values = [metrics_base_source[i+0],metrics_base_target[i+0],metrics_base_target_2[i+0]]
            MAE_values = [metrics_base_source[i+1],metrics_base_target[i+1],metrics_base_target_2[i+1]]
            MAPE_values = [metrics_base_source[i+2],metrics_base_target[i+2],metrics_base_target_2[i+2]]
            R2_score_values = [metrics_base_source[i+3],metrics_base_target[i+3],metrics_base_target_2[i+3]]

            model_names = ['Naive_base_source', 'Naive_base_target', 'Naive_base_target_2']

            metrics_df = pd.DataFrame({
            'Model': model_names,
            'RMSE': [RMSE_values[0], RMSE_values[1], RMSE_values[2]],
            'MAE': [MAE_values[0], MAE_values[1], MAE_values[2]],
            'MAPE': [MAPE_values[0], MAPE_values[1], MAPE_values[2]],
            'R2': [R2_score_values [0], R2_score_values [1], R2_score_values [2]]
            })

        if dataset == '6M':

            RMSE_values = [metrics_base_source[i+0],metrics_base_target[i+0]]
            MAE_values = [metrics_base_source[i+1],metrics_base_target[i+1]]
            MAPE_values = [metrics_base_source[i+2],metrics_base_target[i+2]]
            R2_score_values = [metrics_base_source[i+3],metrics_base_target[i+3]]

            model_names = ['Naive_base_source', 'Naive_base_target']

            metrics_df = pd.DataFrame({
            'Model': model_names,
            'RMSE': [RMSE_values[0], RMSE_values[1]],
            'MAE': [MAE_values[0], MAE_values[1]],
            'MAPE': [MAPE_values[0], MAPE_values[1]],
            'R2': [R2_score_values [0], R2_score_values [1]]
            })

        metrics_df = metrics_df.round(3)

        # Plot the table
        fig, ax = plt.subplots(figsize=(15, 9))  # Set the size as per your requirement
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=metrics_df.values, colLabels=metrics_df.columns, cellLoc='center', loc='center')

        # Adjust vertical space by setting row heights
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 2)  # Scale width by 1, height by 2 (change 2 to a larger value for more height)

        for key, cell in table.get_celld().items():
            cell.set_fontsize(12)  # Adjust font size if needed
            cell.set_linewidth(0.5)  # Adjust line width if needed

        plt.tight_layout()
        if dataset == '12M':
            plt.suptitle('Naive models metrics for next {} predictions in {} datasets'.format(time_period,dataset), y=0.64)
        elif dataset == '6M':
            plt.suptitle('Naive models metrics for next {} predictions in {} datasets'.format(time_period,dataset), y=0.60)
        plt.savefig('Results/Naive/{}/Next {} Results/Metrics Results/metrics_table.png'.format(dataset,time_period), bbox_inches='tight', dpi=500)
        plt.show()


#Results for 12M Datasets


scaler_source = joblib.load('Scalers/12M/source_scaler.pkl')
scaler_target = joblib.load('Scalers/12M/target_scaler.pkl')
scaler_target_2 = joblib.load('Scalers/12M/target_2_scaler.pkl')

with open('Domain sqms/sqm_source.json', 'r') as f:
    sqm_source = json.load(f)

with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)

with open('Domain sqms/sqm_target_2.json', 'r') as f:
    sqm_target_2 = json.load(f)

y_test_source = np.load('Data/Preprocessed data/12M/source_test_y.npy')
y_test_source = y_test_source[:-24]
y_test_target = np.load('Data/Preprocessed data/12M/target_test_y.npy')
y_test_target = y_test_target[:-24]
y_test_target_2 = np.load('Data/Preprocessed data/12M/target_2_test_y.npy')
y_test_target_2 = y_test_target_2[:-24]

predictions_base_source = np.load('Models/Naive/12M/Predictions/predictions_base_source.npy')
predictions_base_target = np.load('Models/Naive/12M/Predictions/predictions_base_target.npy')
predictions_base_target_2 = np.load('Models/Naive/12M/Predictions/predictions_base_target_2.npy')

with open('Models/Naive/12M/Metrics/metrics_base_source.pkl', 'rb') as file:
    metrics_base_source = pickle.load(file)
with open('Models/Naive/12M/Metrics/metrics_base_target.pkl', 'rb') as file:
    metrics_base_target = pickle.load(file)
with open('Models/Naive/12M/Metrics/metrics_base_target_2.pkl', 'rb') as file:
    metrics_base_target_2 = pickle.load(file)


metrics_table('12M')

base_results_graph(predictions_base_source, '12M', 'source')

base_results_graph(predictions_base_target, '12M', 'target')

base_results_graph(predictions_base_target_2, '12M', 'target_2')

error_ranges(predictions_base_target, '12M', 'target')

error_ranges(predictions_base_target_2, '12M', 'target_2')


#Results for 6M Datasets


scaler_target = joblib.load('Scalers/6M/target_scaler.pkl')


with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)


y_test_target = np.load('Data/Preprocessed data/6M/target_test_y.npy')
y_test_target = y_test_target[:-24]


predictions_base_target = np.load('Models/Naive/6M/Predictions/predictions_base_target.npy')
predictions_base_target = predictions_base_target[:-24]

with open('Models/Naive/6M/Metrics/metrics_base_source.pkl', 'rb') as file:
    metrics_base_source = pickle.load(file)
with open('Models/Naive/6M/Metrics/metrics_base_target.pkl', 'rb') as file:
    metrics_base_target = pickle.load(file)


metrics_table('6M')

base_results_graph(predictions_base_target, '6M', 'target')

error_ranges(predictions_base_target, '6M', 'target')
